# DAX + pyadomd + pandas Beispielprojekt

Dieses Notebook zeigt, wie man mit Python, pyadomd und pandas DAX-Abfragen gegen ein Tabular-Modell ausführt und die Ergebnisse verarbeitet. Als Output wird eine excel-Tabelle generiert, die die Anzahl der Neu-Uploads pro Monat auf einer Video-Plattform nach Monat enthält.

⚠️ Hinweis: Der Code ist abstrahiert und dient nur zu Lern- und Demo-Zwecken. Es werden keine echten Verbindungen und keine echten Daten verwenet.

In [ ]:
import pandas as pd
from pyadomd import Pyadomd
from pandas import DataFrame

## Verbindung zum Analysis Services Tabular-Modell aufbauen

Hier wird eine Beispiel-Verbindungszeichenfolge definiert und zwei DAX-Abfragen vorbereitet:
- Vorjahresdaten (bis 2023)
- Daten für 2024


In [ ]:
# Beispielhafte Verbindung (Platzhalter, bitte anpassen)
conn_str = r'Provider=MSOLAP; Data Source=your_server; Catalog=your_catalog;'

# DAX-Abfrage für Vorjahresdaten
query_pre = """
EVALUATE
FILTER(
    SUMMARIZECOLUMNS(
        MyTable[ID],
        MyTable[Item],
        MyTable[Category],
        MyTable[Season],
        MyTable[Episode],
        KEEPFILTERS(FILTER(ALL(DateTable[Year]), DateTable[Year] < 2024)),
        "MetricValue", [SomeMeasure]
    ),
    [SomeMeasure] > 0
)
"""

# DAX-Abfrage für 2024
query_2024 = """
EVALUATE
FILTER(
    SUMMARIZECOLUMNS(
        MyTable[ID],
        MyTable[Item],
        MyTable[Category],
        DateTable[Month],
        MyTable[Season],
        MyTable[Episode],
        KEEPFILTERS(TREATAS({2024}, DateTable[Year])),
        "MetricValue", [SomeMeasure]
    ),
    [SomeMeasure] > 0
)
"""


## DAX-Abfragen ausführen und Daten mit pandas verarbeiten

In [ ]:
with Pyadomd(conn_str) as conn:
    with conn.cursor().execute(query_pre) as cur:
        df_pre = DataFrame(cur.fetchall(), columns=[c.name for c in cur.description])

    with conn.cursor().execute(query_2024) as cur:
        df_2024 = DataFrame(cur.fetchall(), columns=[c.name for c in cur.description])

# Filter: IDs aus 2024, die noch nicht in Vorjahresdaten sind
df_final = df_2024[~df_2024['MyTable[ID]'].isin(df_pre['MyTable[ID]'])].reset_index(drop=True)

# Pro ID den minimalen Monat auswählen
min_indices = df_final.groupby('MyTable[ID]')['DateTable[Month]'].idxmin()
df_final_min = df_final.loc[min_indices].reset_index(drop=True)

# Ergebnis anzeigen
print(df_final_min)

# Ergebnis als Excel speichern
df_final_min.to_excel("df_final_min.xlsx", index=False)
